In [ ]:
# %load house_scaper.py
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import sqlite3
import datetime

df = pd.DataFrame({"MLS": [],"Street": [], "City":[],"ListPrice":[],"Bedrooms":[],"Bathrooms":[],"SqFt":[],"Date":[],  "Price/SqFt":[]})
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October",
              "November", "December"]
month_map = {key: int(val) for key, val in zip(months, range(1, 13))}


def sqlize_string(string):
    return "'" + string + "'"

def get_date(string):
    chunked = string.split()
    return sqlize_string(str(datetime.date(2016, month_map[chunked[0]], int(chunked[1]))))

def scraping(dataframe):
    page = requests.get("http://www.slocountyhomes.com/newlistex.php")
    data = BeautifulSoup(page.text, "html.parser")
    hdrs = ["Bathrooms", "Bedrooms", "City", "Date", "List Price", "MLS", "Price/SqFt", "SqFt", "Street"]
    idx_map = { hdr:idx for hdr, idx in zip(hdrs, range(len(hdrs))) }


    table_rows = data.find_all('tr')
    # print(table_rows)
    listing_date = ""
    for row in table_rows:
        row_entry = [0] * len(hdrs)
        cells = row.find_all("td", recursive=True)
        if len(cells) == 1:
            listing_date = cells[0].text.strip()
            assert listing_date != ""

        elif 0 < len(cells) <= 8 and len(cells) != 3:
            ## CELL ORDER -->   MLS #	Street	City	List Price	Beds	Baths	Sq Footage
            row_entry[idx_map["MLS"]] = int(cells[0].text.strip())
            row_entry[idx_map["Street"]] = sqlize_string(cells[1].text.strip())
            row_entry[idx_map["City"]] = sqlize_string(cells[2].text.strip())
            row_entry[idx_map["List Price"]] = int(cells[3].text.strip()[1:].replace(",", ""))
            row_entry[idx_map["Bedrooms"]] = int(cells[4].text.strip())
            row_entry[idx_map["Bathrooms"]] = int(cells[5].text.strip())
            try: # handle missing Sq footage
                row_entry[idx_map["SqFt"]] = int(cells[6].text.strip())
                row_entry[idx_map["Price/SqFt"]] = row_entry[idx_map["List Price"]] / row_entry[
                    idx_map["SqFt"]]
            except ValueError:
                row_entry[idx_map["SqFt"]] = -1
            row_entry[idx_map["Date"]] = get_date(listing_date)
            ## append this row to dataframe
            # print(row_entry)
            dataframe.loc[len(dataframe)] = row_entry
    ## data integrity
    dataframe.drop(dataframe[dataframe.SqFt == -1].index, inplace=True)
    dataframe.drop_duplicates(inplace=True)
    return dataframe


In [3]:
houses = scraping(df)

In [4]:
houses

,Bathrooms,Bedrooms,City,Date,ListPrice,MLS,Price/SqFt,SqFt,Street
0,2.0,3.0,'Grover Beach','2016-11-20',580000.0,1073069.0,318.156884,1823.0,'917 Savannah'
1,4.0,5.0,'Lompoc','2016-11-20',559000.0,1072903.0,214.012251,2612.0,'2313 Carrizo'
2,2.0,3.0,'Paso Robles','2016-11-20',355000.0,1073067.0,302.385009,1174.0,'633 Laura'
3,3.0,4.0,'San Luis Obispo','2016-11-20',945000.0,1073032.0,363.461538,2600.0,'5640 Pinehurst'
4,3.0,4.0,'Atascadero','2016-11-19',750000.0,1073038.0,258.620690,2900.0,'925 Paseo Pacifico'
5,2.0,3.0,'Los Osos','2016-11-19',569000.0,1073052.0,320.382883,1776.0,'1774 7th'
6,2.0,2.0,'Morro Bay','2016-11-19',430000.0,1073062.0,482.603816,891.0,'3281 Tide'
7,2.0,4.0,'Nipomo','2016-11-19',535000.0,1073057.0,305.017104,1754.0,'255 Cedarwood'
8,3.0,3.0,'Oceano','2016-11-19',749000.0,1073059.0,393.796004,1902.0,'1750 19th'
9,2.0,3.0,'Santa Maria','2016-11-19',325000.0,1073058.0,262.520194,1238.0,'1009 School'


In [6]:
from sqlite_api import SLOHouseDatabase

In [9]:
slo_houses = SLOHouseDatabase()
slo_houses.insert_dataframe(houses)

Connected to slo_housing.db


KeyError: 'List Price'